#Install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dipy

     |████████████████████████████████| 7.7 MB 4.3 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import collections
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import VectorOfEndpointsFeature
from dipy.segment.clustering import QuickBundles
import glob

#Route similarity

##**train**

In [4]:
trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel'

In [5]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'

In [6]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel'

lebeled		      trips_2019_12_5.csv  trips_2019_6_7.csv
trips_2019_10_18.csv  trips_2019_1_29.csv  trips_2019_6_9.csv
trips_2019_10_19.csv  trips_2019_2_25.csv  trips_2019_7_3.csv
trips_2019_10_20.csv  trips_2019_2_4.csv   trips_2019_7_5.csv
trips_2019_11_24.csv  trips_2019_3_27.csv  trips_2019_8_25.csv
trips_2019_11_25.csv  trips_2019_3_30.csv  trips_2019_8_26.csv
trips_2019_11_26.csv  trips_2019_4_23.csv  trips_2019_8_27.csv
trips_2019_1_12.csv   trips_2019_4_24.csv  trips_2019_9_12.csv
trips_2019_12_3.csv   trips_2019_5_18.csv  trips_2019_9_13.csv
trips_2019_12_4.csv   trips_2019_5_20.csv  trips_2019_9_14.csv


In [7]:
all_files = glob.glob('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel' + '/*.csv')

In [8]:
all_files

['/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_8_26.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_8_27.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_8_25.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_9_12.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_9_13.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_9_14.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_11_25.csv',
 '/content/drive/My Drive/2021 Route Prediction/Project-1/Sou

In [9]:
dfs = []
day_len = []
day_trip_count = []
for day in all_files:
  sp_seq_df = pd.read_csv(day, parse_dates=['time_stamp'])
  sp_seq_df = sp_seq_df[sp_seq_df['cluster']>-1].sort_values(by=['trip_id', 'time_stamp'])

  trip_count = sp_seq_df.trip_id.value_counts()
  trip_count = trip_count[trip_count >= 12]
  sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
  sp_seq_df['month'] = sp_seq_df.iloc[0].time_stamp.month
  sp_seq_df['day'] = sp_seq_df.iloc[0].time_stamp.day
  day_len.append(len(sp_seq_df))
  day_trip_count.append(len(trip_count))
  dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


[1334, 1862, 1329, 1808, 1558, 1491, 704, 2597, 1640, 1330, 1704, 1142, 2741, 4066, 1210, 3385, 3688, 2925, 4144, 2121, 2491, 1831, 1515, 1455, 1166, 2282, 1389, 880, 1773]
[78, 108, 78, 89, 81, 80, 36, 144, 75, 69, 95, 62, 128, 193, 59, 205, 192, 134, 225, 105, 130, 100, 76, 82, 71, 107, 66, 45, 79]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,cluster,month,day
0,2019-08-26 09:39:10,02800040000000GBN6103000721,14.964220,102.155282,54,8.0,4,4420,8,26
1,2019-08-26 10:05:42,02800040000000GBN6103000721,15.059160,102.168290,20,8.0,4,177,8,26
2,2019-08-26 11:18:48,02800040000000GBN6103000721,15.272940,102.418368,74,8.0,4,1061,8,26
3,2019-08-26 11:41:48,02800040000000GBN6103000721,15.458613,102.502925,0,8.0,4,211,8,26
4,2019-08-26 12:29:48,02800040000000GBN6103000721,15.873480,102.634000,59,8.0,4,232,8,26
...,...,...,...,...,...,...,...,...,...,...
57556,2019-07-05 02:29:24,013000200000863790021372190,14.838647,101.765426,0,7.0,4544,763,7,5
57557,2019-07-05 02:31:24,013000200000863790021372190,14.838647,101.765427,0,7.0,4544,763,7,5
57558,2019-07-05 03:06:24,013000200000863790021372190,14.751966,102.061578,30,7.0,4544,632,7,5
57559,2019-07-05 03:14:24,013000200000863790021372190,14.738377,102.128649,62,7.0,4544,625,7,5


In [10]:
road_df = pd.read_csv(road_data_path+"/route_network_with_cluster.csv")

cluster_df = road_df[['cluster', 'center_lat', 'center_lon']]
cluster_df = cluster_df.drop_duplicates()
cluster_df.rename(columns = {'cluster':'sp', 'center_lat':'lat', 'center_lon':'lon'}, inplace = True)
cluster_df

,sp,lat,lon
0,0,14.984788,102.051257
156,1,14.979075,102.020065
188,2,14.982676,102.003040
220,3,14.987172,101.986551
244,4,14.992287,101.968000
...,...,...,...
50516,8049,15.765768,101.555882
50518,8050,16.132595,101.584201
50522,8051,15.755759,101.582307
50524,8052,15.557630,101.573573


In [11]:
df.rename(columns = {'cluster':'sp'}, inplace = True)
loc_df = df[['trip_id', 'sp', 'month','day']].merge(cluster_df,how='left')
print(sp_seq_df.isna().any())
loc_df

time_stamp    False
unit_id       False
lat           False
lon           False
speed         False
unit_type     False
trip_id       False
cluster       False
month         False
day           False
dtype: bool


,trip_id,sp,month,day,lat,lon
0,4,4420,8,26,14.964347,102.155420
1,4,177,8,26,15.058801,102.168396
2,4,1061,8,26,15.273002,102.418467
3,4,211,8,26,15.458943,102.503145
4,4,232,8,26,15.873185,102.633850
...,...,...,...,...,...,...
57556,4544,763,7,5,14.838740,101.764770
57557,4544,763,7,5,14.838740,101.764770
57558,4544,632,7,5,14.751802,102.061795
57559,4544,625,7,5,14.738263,102.128775


In [12]:
trips = []
low_12 = 0
days = range(1,32)
months = range(1,13)
for month in months:
  for day in days:
    day_df = loc_df[(loc_df.day == day) & (loc_df.month == month)]
    for t_id in day_df.trip_id.unique():
      trip = day_df[day_df.trip_id == t_id][['lat', 'lon']]
      if len(trip)/12 > 1.5:
        split_trip=[trip.iloc[i:i + 12] for i in range(0, len(trip), 12)]
        if len(split_trip[-1]) < 8:
          last = split_trip.pop()
          before_last = split_trip.pop()
          merge_last = pd.concat([before_last, last])
          split_trip.append(merge_last)
        for partial_trip in split_trip:
          lat_lon_data = np.c_[partial_trip['lat'].values, partial_trip['lon'].values]
          trips.append(lat_lon_data)
      else:
        # print(len(trip))
        lat_lon_data = np.c_[trip['lat'].values, trip['lon'].values]
        trips.append(lat_lon_data)
# print(low_12)
print(len(trips))
trips[0][0,0], trips[0][0,1]

4384


(17.852039619999996, 103.57706924)

In [13]:
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.1'

In [ ]:
import pickle
import collections
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import VectorOfEndpointsFeature
from dipy.segment.clustering import QuickBundles
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=12))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
        currD = np.mean(x)
        return currD

THRESHOLD = 12
metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)

# feature = ResampleFeature(nb_points=24)
# metric = AveragePointwiseEuclideanMetric(feature=feature)
# qb = QuickBundles(threshold=THRESHOLD, metric=metric)

%time clusters = qb.cluster(trips)

pkl_filename = train_path+"/QBcluster_baseline_treshold12.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(clusters, file)
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 10)[True], end=' /')
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

CPU times: user 2h 37min 8s, sys: 21 s, total: 2h 37min 29s
Wall time: 2h 37min 38s
Nb. clusters: 1193
Cluster sizes: <map object at 0x7f15d5064110>
Small clusters: 1173 /Small clusters: [ True  True False ...  True  True  True]
Streamlines indices of the first cluster:
 [0]
Centroid of the last cluster:
 [[ 14.636821  102.76427  ]
 [ 14.6377125 103.181725 ]
 [ 14.638603  103.599174 ]
 [ 14.639495  104.01663  ]
 [ 14.625011  103.761314 ]
 [ 14.606832  103.34425  ]
 [ 14.588652  102.92719  ]
 [ 14.570473  102.51013  ]
 [ 14.617392  102.84563  ]
 [ 14.627858  103.26271  ]
 [ 14.633763  103.68013  ]
 [ 14.6396675 104.09754  ]]


In [14]:
pkl_filename = train_path+"/QBcluster_baseline_treshold12.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(clusters, file)
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 10)[True], end=' /')
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

Nb. clusters: 1193
Cluster sizes: <map object at 0x7efdaecfae50>
Small clusters: 1173 /Small clusters: [ True  True False ...  True  True  True]
Streamlines indices of the first cluster:
 [0]
Centroid of the last cluster:
 [[ 14.636821  102.76427  ]
 [ 14.6377125 103.181725 ]
 [ 14.638603  103.599174 ]
 [ 14.639495  104.01663  ]
 [ 14.625011  103.761314 ]
 [ 14.606832  103.34425  ]
 [ 14.588652  102.92719  ]
 [ 14.570473  102.51013  ]
 [ 14.617392  102.84563  ]
 [ 14.627858  103.26271  ]
 [ 14.633763  103.68013  ]
 [ 14.6396675 104.09754  ]]


In [15]:
loc_df['route_cluster'] = pd.Series()

cluster_list = []
index = 0
for trip in trips:
  lenght = len(trip)
  for cluster in range(len(clusters)):
      if index in clusters[cluster].indices:
        cluster_list.extend([cluster]*lenght)
  index+=1
print(len(cluster_list))
loc_df.route_cluster = cluster_list
loc_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


31045


ValueError: ignored

In [ ]:
df[['day', 'month', 'route_cluster']] = loc_df[['day', 'month', 'route_cluster']]
df = df.drop(['day'], axis=1)
df = df.drop(['month'], axis=1)

In [ ]:
df

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,sp,route_cluster
0,2019-04-23 07:20:30,003000900000000011871803010,15.803121,103.024884,7,8.0,31,5072,0
1,2019-04-23 08:56:23,003000900000000011871803010,16.029287,103.111936,0,8.0,31,1683,0
2,2019-04-23 08:58:23,003000900000000011871803010,16.029287,103.111936,0,8.0,31,1683,0
3,2019-04-23 09:01:23,003000900000000011871803010,16.029287,103.111936,0,8.0,31,1683,0
4,2019-04-23 09:05:23,003000900000000011871803010,16.029287,103.111936,0,8.0,31,1683,0
...,...,...,...,...,...,...,...,...,...
31040,2019-02-04 03:35:31,001000800000000164110108931,14.751941,102.061950,0,8.0,6684,632,70
31041,2019-02-04 03:43:30,001000800000000164110108931,14.751934,102.061940,0,8.0,6684,632,70
31042,2019-02-04 04:51:28,001000800000000164110108931,14.680212,102.554306,56,8.0,6684,601,70
31043,2019-02-04 08:03:04,001000800000000164110108931,14.885179,103.650560,61,8.0,6684,1476,70


In [ ]:
df.to_csv(trips_path + f'/lebeled/testtrips_2019_1-7.csv', encoding='utf-8', index=False)

In [ ]:
print(df.route_cluster.unique())
print(df)

[   0    1    2 ... 1190 1191 1192]
               time_stamp                      unit_id  ...    sp  route_cluster
0     2019-04-23 07:20:30  003000900000000011871803010  ...  5072              0
1     2019-04-23 08:56:23  003000900000000011871803010  ...  1683              0
2     2019-04-23 08:58:23  003000900000000011871803010  ...  1683              0
3     2019-04-23 09:01:23  003000900000000011871803010  ...  1683              0
4     2019-04-23 09:05:23  003000900000000011871803010  ...  1683              0
...                   ...                          ...  ...   ...            ...
31040 2019-02-04 03:35:31  001000800000000164110108931  ...   632             70
31041 2019-02-04 03:43:30  001000800000000164110108931  ...   632             70
31042 2019-02-04 04:51:28  001000800000000164110108931  ...   601             70
31043 2019-02-04 08:03:04  001000800000000164110108931  ...  1476             70
31044 2019-02-04 09:17:46  001000800000000164110108931  ...  1460        